In [1]:
# positive adult
adults_oriented_examples = [
    "Visit the rooftop bar downtown for stunning views and a curated selection of craft cocktails.",
    "Plan a day of wine tasting at the region's top vineyards, complete with private tours.",
    "Book a spa day with massages, saunas, and relaxation areas exclusively for adults.",
    "Attend a live jazz performance at the speakeasy-style venue known for its intimate ambiance.",
    "Join a mixology workshop to learn the secrets behind making perfect cocktails at home.",
    "Take a sunset cruise offering drinks and live music in a serene, adults-only setting.",
    "Visit the cigar lounge downtown, featuring an extensive selection and a cozy atmosphere.",
    "Book a brewery tour with tastings of seasonal beers and behind-the-scenes insights from brewmasters.",
    "Attend an adults-only trivia night at the local pub it's a fun way to test your knowledge.",
    "Visit the distillery for a tour and a guided whiskey tasting experience."
]

# negative child
kids_oriented_examples = [
    "Visit the interactive science museum with hands-on exhibits perfect for kids of all ages.",
    "Plan a day at the local zoo, featuring kid-friendly animal encounters and feeding sessions.",
    "Explore the city's largest playground with climbing structures, slides, and picnic areas for families.",
    "Spend time at the local aquarium, known for its touch tanks and playful sea otters.",
    "Visit the city's amusement park with rides designed specifically for younger kids and toddlers.",
    "Head to the local library for storytime sessions and engaging activities for kids.",
    "Visit the butterfly garden it's colorful, educational, and a hit with younger children.",
    "Take a scenic train ride that offers kid-friendly entertainment and stunning views along the way.",
    "Visit the local zookeeper for a behind-the-scenes tour kids love learning about animals up close.",
    "Spend time at the nearby petting farm, where kids can feed and interact with friendly animals.",
    "Explore the city's famous ice cream parlor, offering fun flavors and mini cones for little ones."
]

In [2]:
import easysteer.hidden_states as hs
from vllm import LLM
import os
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/",
    task="reward",  # Use reward task to get hidden states
    tensor_parallel_size=1
)

all_hidden_states, outputs = hs.get_all_hidden_states(llm, adults_oriented_examples+kids_oriented_examples)

INFO 07-17 00:20:22 [__init__.py:244] Automatically detected platform cuda.
INFO 07-17 00:20:36 [config.py:1472] Using max model len 32768
INFO 07-17 00:20:37 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-17 00:20:37 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-17 00:20:40 [default_loader.py:272] Loading weights took 0.96 seconds
INFO 07-17 00:20:40 [model_runner.py:1255] Model loading took 2.8876 GiB and 1.155259 seconds


Adding requests:   0%|          | 0/21 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/21 [00:00<?, ?it/s, est. speed input: 0.00 toks/s,

In [5]:
from easysteer.steer import extract_linear_probe_control_vector, StatisticalControlVector
control_vector = extract_linear_probe_control_vector(
    all_hidden_states=all_hidden_states, 
    positive_indices=list(range(10)), 
    negative_indices=list(range(10,20)), 
    model_type="qwen2.5",
    token_pos=-1, 
    normalize=True
)
control_vector.export_gguf("style-probe.gguf")

Computing LinearProbe directions:   0%|          | 0/28 [00:00<?, ?it/s]